In [ ]:
import random
from datetime import datetime, timedelta
from database import SessionLocal
from models import Car, User, Tyre, Service, RegisterCar, Orders, OrdersDetail, Appointment
from datetime import datetime


# Create a new session
session = SessionLocal()

"""
1. Generate 1,000 User Accounts.
2, Register a car for the user.
2. Automate 100,000 Transactions.
"""


def get_random_date(
    start_date: str = "01-01-2018",
    end_date: str = "24-11-2024",
    date_format: str = "%d-%m-%Y"
) -> str:
    """
    Generate a random date between two specified dates.

    Parameters:
    - start_date (str): The start date in string format. Defaults to "01-01-2018".
    - end_date (str): The end date in string format. Defaults to "24-11-2024".
    - date_format (str): The date format to parse the input dates.

    Returns:
    - str: Random date between the start and end dates, formatted as 'YYYY-MM-DD'.
    """
    # Convert strings to datetime objects
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)

    # Calculate the range in days between start and end
    delta_days = (end - start).days

    # Generate a random number of days within the range
    random_days = random.randint(0, delta_days)

    # Generate the random date
    random_date = start + timedelta(days=random_days)

    # Return date in 'YYYY-MM-DD' format
    return random_date.strftime("%Y-%m-%d")


def get_random_time() -> str:
    """
    Generate a random time between 09:00 and 16:00, with minutes always set to :00.

    Returns:
    - str: Random time in 'HH:MM' (24-hour) format.
    """
    # Generate a random hour between 9 and 16 (inclusive)
    random_hour = random.randint(9, 16)

    # Format the time as 'HH:00'
    random_time = f"{random_hour:02d}:00"

    return random_time

In [ ]:
from faker import Faker
import random
import string
from passlib.context import CryptContext
import uuid


all_cars = [[car.car_brand, car.car_model, car.tyre_size, car.car_year, car.car_type]
            for car in session.query(Car).all()]

# Initialize Faker instance
fake = Faker('en_MS')
bcrypt_context = CryptContext(schemes=["bcrypt"], deprecated="auto")
lst_email = []

for i in range(250):
    email = fake.email()  # Generate a random email address

    if email in lst_email:
        email = fake.email()
        lst_email.append(email)
    else:
        lst_email.append(email)

    # Generate a random 12-character password
    password = '123456'
    first_name = fake.first_name()  # Generate a random first name
    last_name = fake.last_name()  # Generate a random last name'





    user = User(accountid=str(uuid.uuid4()), email=email, password=bcrypt_context.hash(password),
                firstname=first_name, lastname=last_name, username=email, createdat= get_random_date() + " " + get_random_time())
    
    session.add(user)
    session.commit()

    plate_number = fake.license_plate()
    plate_number = plate_number.replace(" ", "").lower().replace("-", "")

    for k in range(random.randint(1, 3)):

        selected_car = random.choice(all_cars)
        car = RegisterCar(platenumber= plate_number, carbrand=selected_car[0], carmodel=selected_car[1],
                tyresize= selected_car[2], caryear=selected_car[3], accountid=user.accountid, cartype=selected_car[4], createdat=datetime.now())
        session.add(car)
        session.commit()

    print(f"User {i} created")




# Generate random data

In [ ]:
new_user = User(accountid=str(uuid.uuid4()), isAdmin='Y', email="rahmanrom@gmail.com", password=bcrypt_context.hash("123456"),
                firstname="Rahman", lastname="Rom", username="rahmanrom@gmail.com", createdat=get_random_date() + " " + get_random_time())
session.add(new_user)
session.commit()

two_plates = ["wyd1234", "wyd1235"]

for plate in two_plates:
    selected_car = random.choice(all_cars)
    car = RegisterCar(platenumber=plate, carbrand=selected_car[0], carmodel=selected_car[1],
                      tyresize=selected_car[2], caryear=selected_car[3], accountid=new_user.accountid, cartype=selected_car[4], createdat=datetime.now())
    session.add(car)
    session.commit()

In [4]:
tyre_ids = [tyre.itemid for tyre in session.query(Tyre).all()]
service_ids = [service.serviceid for service in session.query(Service).all()]
user_ids = [user.accountid for user in session.query(User).all()]
print(user_ids)

for i in range(2843):
    random_user_id = random.choice(user_ids)
    lst_car = [car.carid for car in session.query(RegisterCar).filter(RegisterCar.accountid == random_user_id).all()]
    selected_car_id = random.choice(lst_car)
    random_number_of_tyres = random.randint(1, 2)
    random_tyre_ids = random.sample(tyre_ids, random_number_of_tyres)
    random_service_id = random.choice(service_ids)

    """
    1. Create Order First
    2. Create Order Details
    3. Create Order Service
    """


    user_details = session.query(User).filter(User.accountid == random_user_id).first()
    date_created = user_details.createdat
    print(date_created)
    convert_date_to_string = date_created.strftime("%d-%m-%Y")

    order_id = str(uuid.uuid4())
    appointment_date = get_random_date(start_date=convert_date_to_string)
    appointment_time = get_random_time()
    appointment_bay = random.randint(1, 5)

    # Create Order
    order = Orders(
        orderid = order_id,
        accountid = random_user_id,
        createdat = appointment_date + ' ' + appointment_time,
        paymentmethod = random.choice(['Cash', 'Card', 'E-Wallet']),

    )

    session.add(order)
    session.commit()

    for tyre_id in random_tyre_ids:
        random_quantity = random.randint(1, 3)
        unit_price = session.query(Tyre).filter(
                Tyre.itemid == tyre_id).first().unitprice
        order_detail = OrdersDetail(
            orderid = order_id,
            productid = tyre_id,
            quantity = random_quantity,
            totalprice= unit_price * random_quantity,
            unitprice = unit_price
        )

        session.add(order_detail)
        session.commit()

    add_service = OrdersDetail(
        orderid = order_id,
        productid = random_service_id,
        quantity = 1,
        totalprice = session.query(Service).filter(Service.serviceid == random_service_id).first().price,
        unitprice = session.query(Service).filter(Service.serviceid == random_service_id).first().price
    )

    session.add(add_service)
    session.commit()
    
    appointment_id = str(uuid.uuid4())
    appointment = Appointment(
        appointmentid = appointment_id,
        accountid = random_user_id,
        appointmentdate = appointment_date + ' ' + appointment_time,
        createdat = datetime.now(),
        status = 'Future',
        appointment_bay = appointment_bay,
        carid = selected_car_id
    )

    session.add(appointment)
    session.commit()

    # Update the Order ID for the new appointment
    appointment.orderid = order_id

    # Calculate the total price for the appointment
    total_price = sum(detail.totalprice for detail in session.query(OrdersDetail).filter(OrdersDetail.orderid == order_id).all())

    # Update the total price in the Orders table
    order.totalprice = total_price
    order.appointmentid = appointment_id

    session.commit()

['33d55ced-abb4-40a5-914a-144d6797d5d7', '773a5d16-47e0-4529-8a14-fa8a0de754f0', 'b5007908-355f-488f-9ca0-e85ac26b9a0d', '447b9192-ed38-40ec-8c05-4984f18c0fb2', '2e7d435f-d47b-49af-bda9-ec2acc219465', 'd8a0adca-c631-426d-a37a-999465ce6dd1', '3d94a8cc-0333-438d-8d86-fdb90852e62f', 'e6a830a5-0038-4ceb-8625-abe0ff5ca285', '0c4c834d-e848-45b4-9fba-fe4c97f0fd74', '83440c98-8143-4c94-ad8d-3e716a0215dc', '6b07708a-117f-434f-b035-0fdaef61a29c', '444db031-57ad-4e02-a579-f2b89b627e94', 'd7b2d435-fb59-4a7a-b3ea-bd5cc88a9914', '539f9155-fd7b-4cb0-844d-e8174d10f8eb', '2a8693ba-a936-4864-ba84-5b99688674c6', '5ee2f985-7aa5-4f43-9ed0-e244db6d365e', 'f175f255-cc38-4ee0-a817-2bf07bf46191', '75f29139-7415-46f1-bb88-5d3b95784181', '9a5ca4dc-6d29-4991-8367-2a4720e645a8', '481ba043-6a1a-4cae-9e80-c56c743306fe', 'f171de59-ebc3-405a-8ec8-817f810205f8', 'b2f4019f-06df-419f-8095-ff876f181cfd', '2bd7b696-b106-463b-a3d8-0c7de48a2867', '3461ede5-43a1-4805-913e-98189c19cf19', '1ca294e7-7881-4dcf-83ad-a518a6a40a84',

In [ ]:
itemid = 'T001'
tyre = session.query(Tyre).filter(Tyre.itemid == itemid).first()
if tyre:
    price = tyre.unitprice
else:
    price = None

print(price)



In [ ]:


for x in range(10):
    print(f"Random Date {x}: {get_random_date()}")
    print(f"Random Time {x}: {get_random_time()}")
